In [1]:
import pandas as pd
import numpy as np
import os as os
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import acessos as ac
import funcionalidade as fc
import re
from pylift import TransformedOutcome
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Obtenção de dados - fonte da tabela na pasta sql

In [2]:
query = """
  
select * from customer_sandbox_zone.dados_estudo_mgm_flavia

"""

df = ac.df_athena('flavia-costa', query)

In [3]:
len(df)

1035884

### Definindo tipos de var e lidando com missings

In [4]:
#variáveis para recodificar como zero
df['share_of_wallet'] = df['share_of_wallet'].fillna(0.0)
df['ganho_limite'] = df['ganho_limite'].fillna(0.0)

df['nr_purchases_online'] = df['nr_purchases_online'].fillna(0.0)
df['nr_acessos'] = df['nr_acessos'].fillna(0.0)
df['pc_acessos_lojawill'] = df['pc_acessos_lojawill'].fillna(0.0)
df['nr_chaves_importantes'] = df['nr_chaves_importantes'].fillna(0.0)

In [5]:
id_columns = ['cpf', 'ordem']
cat_columns = df.drop(id_columns, axis = 1).select_dtypes('object').columns
num_columns = df.drop(id_columns, axis = 1).select_dtypes(exclude=['object']).columns

In [6]:
for column in df.columns:
  if df[column].dtype == 'object':
    df[column].fillna(df[column].mode()[0], inplace = True)
  elif df[column].dtype == 'string':
    df[column].fillna(df[column].mode()[0], inplace = True)
  elif df[column].dtype != 'object':
    df[column].fillna(df[column].median(), inplace = True)

In [7]:
df.isnull().sum()

cpf                         0
tempo_aprovacao             0
tempo_relacionamento        0
ganho_limite                0
ds_risk_type                0
gender                      0
nr_declared_income          0
ds_region                   0
flag_autonomo               0
max_limite                  0
max_qtd_if                  0
avg_utilizado_interno       0
sum_utilizado_interno       0
avg_utilizado_mercado       0
sum_utilizado_mercado       0
nr_chaves_importantes       0
nr_antecipacao_fatura       0
nr_bolso_ativo              0
nr_trans_mov_conta          0
nr_purchases_virtual        0
avg_vl_total_spending       0
nr_purchases_online         0
nr_acessos                  0
pc_acessos_lojawill         0
flag_mgm                    0
dias_atraso                 0
ordem                       0
flag_limite_insuficiente    0
share_of_wallet             0
dtype: int64

In [8]:
df['flag_mgm'].value_counts()

0.0    1000000
1.0      35884
Name: flag_mgm, dtype: int64

## Início modelagem

In [9]:
#tempo_relacionamento, nr_acessos, share of wallet, declared income,  purchases on line, max_limite, Avg Spending, trans mov conta, limite insuficiente, ganho limite, percentual loja will
preditores = ['tempo_aprovacao', 'flag_autonomo', 'dias_atraso', 'tempo_relacionamento', 'avg_vl_total_spending', 'nr_declared_income', 'ganho_limite', 'flag_limite_insuficiente', 'share_of_wallet', 'nr_trans_mov_conta', 'nr_purchases_online']
target = ['flag_mgm']

In [10]:
def discretiza(df, drop_col):
    num_columns = df.drop(drop_col, axis = 1).select_dtypes(exclude=['object']).columns
    stats = df.drop(drop_col, axis=1).describe()
    for i in num_columns:
        p75 = stats.iloc[6][i]
        p25 = stats.iloc[4][i]
        p50 = stats.iloc[5][i]
        
        df['aux'] = np.where((df[i] <= p25), "A - " + i,
                    np.where((df[i] > p25) & (df[i] <= p50), "B - " + i,
                    np.where((df[i] > p50) & (df[i] <= p75), "C - " + i, "D - " + i)))
  
        df = df.rename(columns = {'aux':i+"_fx"})

    return df

In [11]:
df = discretiza(df, ['cpf', 'flag_mgm', 'ordem'])

In [12]:
df['seg_online'] = np.where(df.nr_purchases_online <=  df.nr_purchases_online.quantile(0.25), 'A', 
                    np.where(df.nr_purchases_online <=  df.nr_purchases_online.quantile(0.75), 'B', 'C'                       
                            ))

df['seg_online'].value_counts()

B    520595
A    259292
C    255997
Name: seg_online, dtype: int64

In [13]:
df[['seg_online', 'flag_mgm']].groupby('seg_online').mean()

,flag_mgm
seg_online,
A,0.020957
B,0.046660
C,0.024059


In [14]:
df['seg_mov_conta'] = np.where(df.nr_trans_mov_conta <=  df.nr_trans_mov_conta.quantile(0.75), 'A', 'B')

df[['seg_mov_conta', 'flag_mgm']].groupby('seg_mov_conta').agg({'flag_mgm':['mean', 'count']})

flag_mgm        
                   mean   count
seg_mov_conta                  
A              0.024382  785008
B              0.066742  250876

In [15]:
df['seg_share_of_wallet'] = np.where(df.share_of_wallet <=  df.share_of_wallet.quantile(0.25), 'A', 'B')

df[['seg_share_of_wallet', 'flag_mgm']].groupby('seg_share_of_wallet').agg({'flag_mgm':['mean', 'count']})

flag_mgm        
                         mean   count
seg_share_of_wallet                  
A                    0.004549  258971
B                    0.044672  776913

In [16]:
df['seg_ganho_limite'] = np.where(df.ganho_limite <=  df.ganho_limite.quantile(0.25), 'A',
                                  np.where(df.ganho_limite <=  df.ganho_limite.quantile(0.75), 'B', 'C'))

df[['seg_ganho_limite', 'flag_mgm']].groupby('seg_ganho_limite').agg({'flag_mgm':['mean', 'count']})

flag_mgm        
                      mean   count
seg_ganho_limite                  
A                 0.016087  528641
B                 0.077947  249426
C                 0.030789  257817

In [17]:
df['seg_renda_declarada'] = np.where(df.nr_declared_income <=  df.nr_declared_income.quantile(0.50), 'A', 'B')

df[['seg_renda_declarada', 'flag_mgm']].groupby('seg_renda_declarada').agg({'flag_mgm':['mean', 'count']})

flag_mgm        
                         mean   count
seg_renda_declarada                  
A                    0.019189  544280
B                    0.051749  491604

In [18]:
df['seg_relacionamento'] = np.where(df.tempo_relacionamento <=  df.tempo_relacionamento.quantile(0.25), 'A',
                                  np.where(df.tempo_relacionamento <=  df.tempo_relacionamento.quantile(0.75), 'B', 'C'))

df[['seg_relacionamento', 'flag_mgm']].groupby('seg_relacionamento').agg({'flag_mgm':['mean', 'count']})

flag_mgm        
                        mean   count
seg_relacionamento                  
A                   0.000015  259593
B                   0.065912  518765
C                   0.006551  257526

In [19]:
df['seg_spending'] = np.where(df.avg_vl_total_spending <=  df.avg_vl_total_spending.quantile(0.50), 'A', 'B')

df[['seg_spending', 'flag_mgm']].groupby('seg_spending').agg({'flag_mgm':['mean', 'count']})

flag_mgm        
                  mean   count
seg_spending                  
A             0.030014  579005
B             0.040505  456879

In [20]:
df['seg_tempo_aprovacao'] = np.where(df.tempo_aprovacao <=  df.tempo_aprovacao.quantile(0.25), 'A', 'B')

df[['seg_tempo_aprovacao', 'flag_mgm']].groupby('seg_tempo_aprovacao').agg({'flag_mgm':['mean', 'count']})

flag_mgm        
                         mean   count
seg_tempo_aprovacao                  
A                    0.029612  418551
B                    0.038051  617333

In [21]:
df['seg_autonomo'] = np.where(df.flag_autonomo == 0, 'A', 'B')
df['seg_atraso'] = np.where(df.dias_atraso == 0, 'A', 'B')

In [25]:
df['segmento_v1'] = df.seg_relacionamento.astype("string") + df.seg_spending.astype("string")  + df.seg_share_of_wallet.astype("string") 

'''
\
+ df.seg_mov_conta.astype("string") + df.seg_online.astype("string") + df.seg_ganho_limite.astype("string") + df.seg_renda_declarada.astype("string") \
+ df.seg_tempo_aprovacao.astype("string")

'''

'\n+ df.seg_mov_conta.astype("string") + df.seg_online.astype("string") + df.seg_ganho_limite.astype("string") + df.seg_renda_declarada.astype("string") + df.seg_tempo_aprovacao.astype("string")\n\n'

In [26]:
df[['segmento_v1', 'flag_mgm']].groupby('segmento_v1').agg({'flag_mgm':['mean', 'count']})

flag_mgm        
                 mean   count
segmento_v1                  
AAA          0.000024   82859
AAB          0.000015   66367
ABA          0.000000   34171
ABB          0.000013   76196
BAA          0.014091   60960
BAB          0.070840  229193
BBA          0.021116   10087
BBB          0.077268  218525
CAA          0.000896   61375
CAB          0.002875   78251
CBA          0.005148    9519
CBB          0.012530  108381

In [30]:
218525 / len(df)

0.21095508763529508

In [31]:
76681 / (1000000 + 35884)

0.07402469774607967

In [32]:
201718 / (1000000 + 35884)

0.19473029798703329

In [27]:
df['segmento_v2'] = df.seg_relacionamento.astype("string") + df.seg_spending.astype("string")  + df.seg_share_of_wallet.astype("string") + df.seg_mov_conta.astype("string")

'''
\
+ df.seg_mov_conta.astype("string") + df.seg_online.astype("string") + df.seg_ganho_limite.astype("string") + df.seg_renda_declarada.astype("string") \
+ df.seg_tempo_aprovacao.astype("string")

'''

'\n+ df.seg_mov_conta.astype("string") + df.seg_online.astype("string") + df.seg_ganho_limite.astype("string") + df.seg_renda_declarada.astype("string") + df.seg_tempo_aprovacao.astype("string")\n\n'

In [29]:
df[['segmento_v2', 'flag_mgm']].groupby('segmento_v2').agg({'flag_mgm':['mean', 'count']})

flag_mgm        
                 mean   count
segmento_v2                  
AAAA         0.000025   80512
AAAB         0.000000    2347
AABA         0.000018   54059
AABB         0.000000   12308
ABAA         0.000000   32342
ABAB         0.000000    1829
ABBA         0.000018   56470
ABBB         0.000000   19726
BAAA         0.012154   58251
BAAB         0.055740    2709
BABA         0.059045  157016
BABB         0.096499   72177
BBAA         0.017589    8471
BBAB         0.039604    1616
BBBA         0.062500  127953
BBBB         0.098132   90572
CAAA         0.000739   60860
CAAB         0.019417     515
CABA         0.002329   63988
CABB         0.005328   14263
CBAA         0.003754    9057
CBAB         0.032468     462
CBBA         0.010299   76029
CBBB         0.017773   32352

O segmento_v2 ABA, no consolidado tinha share de 0.038194


Na abertura de share of wallet
B = 3
A = 1





# variaveis finais


df['seg_relacionamento'] = np.where(df.tempo_relacionamento <=  df.tempo_relacionamento.quantile(0.25), 'A',
                                  np.where(df.tempo_relacionamento <=  df.tempo_relacionamento.quantile(0.75), 'B', 'C'))




seg_relacionamento
seg_mov_conta
seg_share_of_wallet
seg_ganho_limite
segmento_v2 = df.seg_ganho_limite.astype("string") + df.seg_relacionamento.astype("string") + df.seg_mov_conta.astype("string")


In [67]:
df['segmento_v5'] = np.where(df.seg_relacionamento == "A", "0. Pouco tempo de relacionamento",
np.where(df.seg_relacionamento.astype("string") + df.seg_mov_conta.astype("string") == "CA", "0. Alto Tempo de Relac - Pouco Movimento Conta",
#Abrir por share of wallet
np.where((df.segmento_v2 == "ABA") & (df.seg_share_of_wallet == "A"), "1. Tempo de Relac Mediano - Pouco aumento limite - Pouco Mov Conta - Baixo SOW", 
np.where((df.segmento_v2 == "ABA") & (df.seg_share_of_wallet == "B"), "3. Tempo de Relac Mediano - Pouco aumento limite - Pouco Mov Conta - Alto SOW",
np.where(df.segmento_v2 == "ABB", "4. Tempo de Relac Mediano - Pouco aumento limite - Alto Movimento Conta",
np.where(df.segmento_v2 == "BBB", "5. Tempo de Relac Mediano - Aumento limite mediano - Alto Movimento Conta",
np.where(df.segmento_v2 == "ACB", "1. Alto Tempo de Relac - Pouco aumento limite - Alto Movimento Conta",
np.where(df.segmento_v2 == "CCB", "1. Alto Tempo de Relac - Alto aumento de limite - Alto Movimento Conta",
np.where(df.segmento_v2 == "BCB", "0. Alto Tempo de Relac - Aumento limite mediano - Alto Movimento Conta",
np.where(df.segmento_v2 == "BBA", "4. Tempo de Relac Mediano - Aumento limite mediano - Pouco Movimento Conta",
np.where(df.segmento_v2 == "CBA", "2. Tempo de Relac Mediano - Aumento limite mediano - Baixo Movimento Conta",
np.where(df.segmento_v2 == "CBB", "3. Tempo de Relac Mediano - Alto aumento limite - Alto Movimento Conta", "NI"))))))))))))

df[['segmento_v5', 'flag_mgm']].groupby('segmento_v5').agg({'flag_mgm':['mean', 'count']})

flag_mgm        
                                                        mean   count
segmento_v5                                                         
0. Alto Tempo de Relac - Aumento limite mediano...  0.008523    4576
0. Alto Tempo de Relac - Pouco Movimento Conta      0.004816  209934
0. Pouco tempo de relacionamento                    0.000015  259593
1. Alto Tempo de Relac - Alto aumento de limite...  0.018450   23252
1. Alto Tempo de Relac - Pouco aumento limite -...  0.010524   19764
1. Tempo de Relac Mediano - Pouco aumento limit...  0.012658   48034
2. Tempo de Relac Mediano - Aumento limite medi...  0.035810  113208
3. Tempo de Relac Mediano - Alto aumento limite...  0.053479   55498
3. Tempo de Relac Mediano - Pouco aumento limit...  0.056947   65412
4. Tempo de Relac Mediano - Aumento limite medi...  0.077881  125037
4. Tempo de Relac Mediano - Pouco aumento limit...  0.099986   34895
5. Tempo de Relac Mediano - Aumento limite medi...  0.125337   76681

In [68]:
df[['segmento_v5', 'tempo_relacionamento']].groupby('segmento_v5').agg({'tempo_relacionamento':['mean', 'min', 'max']})

tempo_relacionamento  \
                                                                   mean   
segmento_v5                                                               
0. Alto Tempo de Relac - Aumento limite mediano...          1504.578671   
0. Alto Tempo de Relac - Pouco Movimento Conta              1714.447522   
0. Pouco tempo de relacionamento                              94.263262   
1. Alto Tempo de Relac - Alto aumento de limite...          1529.185618   
1. Alto Tempo de Relac - Pouco aumento limite -...          1865.712862   
1. Tempo de Relac Mediano - Pouco aumento limit...           721.869322   
2. Tempo de Relac Mediano - Aumento limite medi...           883.816577   
3. Tempo de Relac Mediano - Alto aumento limite...           863.985189   
3. Tempo de Relac Mediano - Pouco aumento limit...           599.984315   
4. Tempo de Relac Mediano - Aumento limite medi...           613.933516   
4. Tempo de Relac Mediano - Pouco aumento limit...           580.665654   
5. Tempo de Relac Mediano - Aumento limite medi...           585.395378   

                                                                    
                                                       min     max  
segmento_v5                                                         
0. Alto Tempo de Relac - Aumento limite mediano...  1346.0  2260.0  
0. Alto Tempo de Relac - Pouco Movimento Conta      1346.0  2309.0  
0. Pouco tempo de relacionamento                       1.0   218.0  
1. Alto Tempo de Relac - Alto aumento de limite...  1346.0  2308.0  
1. Alto Tempo de Relac - Pouco aumento limite -...  1346.0  2308.0  
1. Tempo de Relac Mediano - Pouco aumento limit...   219.0  1345.0  
2. Tempo de Relac Mediano - Aumento limite medi...   219.0  1345.0  
3. Tempo de Relac Mediano - Alto aumento limite...   219.0  1345.0  
3. Tempo de Relac Mediano - Pouco aumento limit...   219.0  1345.0  
4. Tempo de Relac Mediano - Aumento limite medi...   219.0  1345.0  
4. Tempo de Relac Mediano - Pouco aumento limit...   219.0  1345.0  
5. Tempo de Relac Mediano - Aumento limite medi...   219.0  1345.0